In [3]:
!pip install bertopic==0.12.0

  Using cached bertopic-0.12.0-py2.py3-none-any.whl (90 kB)
  Using cached hdbscan-0.8.29-cp39-cp39-linux_x86_64.whl
  Using cached umap_learn-0.5.3-py3-none-any.whl
  Using cached PyYAML-5.4.1-cp39-cp39-manylinux1_x86_64.whl (630 kB)
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached Cython-0.29.32-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (2.0 MB)
  Using cached transformers-4.25.1-py3-none-any.whl (5.8 MB)
  Using cached sentencepiece-0.1.97-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached huggingface_hub-0.11.1-py3-none-any.whl (182 kB)
  Using cached pynndescent-0.5.8-py3-none-any.whl
  Using cached numba-0.56.4-cp39-cp39-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.5 MB)
  Using cached llvmlite-0.39.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.6 MB)
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [4]:
# Til dataen
import pandas as pd
import seaborn as sns
import numpy as np

# Til BERTopic
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

# Til at gemme variabler
import pickle

# Til tokenizer 
#!pip install spacy
#!python -m spacy download en_core_web_sm

import spacy
from spacy.lang.en import English

# Natural language processing
import nltk, re, pprint
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-03 23:01:41.967894: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-03 23:01:42.079243: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-03 23:01:42.083766: W tensorflow/stream_executor/platform/default/dso_loader.

In [6]:
# Importer data
df = pd.read_csv("/work/Topic assignment/UN_blocs_EW.csv")

In [7]:
#Fjern skrald og tokenize sætninger
def make_sentences(text):
    text = text.replace('"', '' )
    text = text.replace("'","")
    text = text.replace("\ufeff", "")
    text = text.replace("\n", "")
    #text = text.replace(".", ". ")
    #Tokenize text by sentence, ready for translation
    speech_sent = list(sent_tokenize(text))
    return speech_sent

df.text = df.text.apply(lambda x: make_sentences(x))

In [8]:
# 'Explode' hver række til en række per sætning
df = df.explode("text", ignore_index=False)
df = df.reset_index(drop=False)
df.rename(columns = {'index':'speech ID'}, inplace = True)
df.text = df.text.apply(str)

In [9]:
stopwords = list(stopwords.words('english'))
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords)

In [10]:
umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)
hdbscan_model = HDBSCAN(min_cluster_size=80, min_samples=40,
                        gen_min_span_tree=True,
                        prediction_data=True)

In [ ]:
# DEN HER TAGER 25 MINUTTER AT KØRE. FJERN KOMMENTAR-TEGN PÅ EGET ANSVAR!

'''
model = BERTopic(
    umap_model = umap_model, 
    hdbscan_model = hdbscan_model,
    embedding_model="all-MiniLM-L6-v2",
    vectorizer_model=vectorizer_model,
    top_n_words=10,
    language='english',
    calculate_probabilities=True,
    verbose=True
)

bloc_topics, probs = model.fit_transform(list(df.text))

# ------------ Save the model ------------
with open('/work/Topic assignment/topics_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# -------- Save the bloc topics ----------
with open('/work/Topic assignment/bloc_topics.pkl', 'wb') as f:
    pickle.dump(bloc_topics, f)

# ------- Save the probabilities ---------
with open('/work/Topic assignment/probs.pkl', 'wb') as f:
    pickle.dump(probs, f)
    
print("why did you run me?? I told you not to run me >:(")

SyntaxError: EOF while scanning triple-quoted string literal (3286782903.py, line 29)

In [11]:
# ------------ Load the model ------------
with open('/work/Topic assignment/topics_model.pkl', 'rb') as f:
    model = pickle.load(f)

# -------- Load the bloc topics ----------
with open('/work/Topic assignment/bloc_topics.pkl', 'rb') as f:
    bloc_topics = pickle.load(f)

# ------- Load the probabilities ---------
with open('/work/Topic assignment/probs.pkl', 'rb') as f:
    probs = pickle.load(f)

In [ ]:
freq = model.get_topic_info()
print("Number of topics: {}".format(len(freq)))
freq

Number of topics: 164


,Topic,Count,Name
0,-1,40440,-1_world_international_countries_states
1,0,4943,0_united nations_nations_united_charter
2,1,4414,1_israel_middle_middle east_arab
3,2,3369,2_africa_south_south africa_namibia
4,3,1704,3_detente_tension_conflicts_world
...,...,...,...
159,158,83,158_korea_south korea_foreign troops_troops
160,159,82,159_commodity_prices_commodities_price
161,160,81,160_facts_truth_question_facts question
162,161,81,161_drought_pakistan_refugees_afghan


In [ ]:
model.visualize_barchart()

In [ ]:
model.visualize_topics()

In [ ]:
hierarchical_topics = model.hierarchical_topics(list(df.text))
model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

100%|██████████| 162/162 [00:02<00:00, 65.35it/s]


In [ ]:
topics_per_class = model.topics_per_class(list(df.text), classes=list(df.block))
fig = model.visualize_topics_per_class(topics_per_class, topics=[15, 21, 31, 77, 50, 156, 100])
fig.write_html("/work/economic_class.html")

2it [00:03,  1.96s/it]


In [ ]:
topics_over_time = model.topics_over_time(list(df.text), list(df.year))
fig = model.visualize_topics_over_time(topics_over_time, topics=[15, 21, 31, 77, 50, 156, 100])
fig.write_html("/work/economic_dynamic.html")

22it [03:47, 10.33s/it]


In [ ]:
model.get_topic(100)

[('economic', 0.0339603445129321),
 ('international economic', 0.02956253006894773),
 ('special session', 0.02816434806378479),
 ('charter economic', 0.026527071447146333),
 ('duties states', 0.02558514350348568),
 ('rights duties', 0.02492367377407705),
 ('economic rights', 0.024723793197117643),
 ('special', 0.023930668134844266),
 ('sixth special', 0.022930434473085296),
 ('economic order', 0.02284400903934996)]

In [12]:
similar_topics, similarity = model.find_topics("afghanistan", top_n=10)

In [ ]:
model.get_topic(similar_topics[0])

[('afghanistan', 0.054747414206054244),
 ('afghan', 0.025138553596630427),
 ('republic afghanistan', 0.020474819237869524),
 ('afghan people', 0.015359031546145756),
 ('pakistan', 0.011848442748987296),
 ('democratic republic', 0.010901031104076133),
 ('democratic', 0.010801702066917345),
 ('people', 0.009451511830634543),
 ('republic', 0.009314140380322203),
 ('soviet', 0.008252497964629969)]

In [13]:
topics_per_class = model.topics_per_class(list(df.text), classes=list(df.block))
fig = model.visualize_topics_per_class(topics_per_class, topics=[similar_topics[0], similar_topics[1], similar_topics[2], similar_topics[3], similar_topics[4], similar_topics[5], similar_topics[6], similar_topics[7], similar_topics[8], similar_topics[9]])
fig.write_html("/work/afghanstian_class.html")

2it [00:03,  1.97s/it]


In [14]:
topics_per_class = model.topics_per_class(list(df.text), classes=list(df.group))
fig = model.visualize_topics_per_class(topics_per_class, topics=[similar_topics[0], similar_topics[1], similar_topics[8]])
fig.write_html("/work/afghanstian_all_class.html")

5it [00:05,  1.20s/it]


In [15]:
topics_over_time = model.topics_over_time(list(df.text), list(df.year))
fig = model.visualize_topics_over_time(topics_over_time, topics=[similar_topics[0], similar_topics[1], similar_topics[8]])
fig.write_html("/work/afghanstian_dynamic.html")

22it [03:47, 10.36s/it]


Development ikke længere økonomisk?

In [ ]:
topics_per_class = model.topics_per_class(list(df.text), classes=list(df.block))
fig = model.visualize_topics_per_class(topics_per_class, topics=[37, 86, 18, 67, 82])
fig.write_html("/work/cultural_class.html")

2it [00:03,  1.99s/it]


In [ ]:
topics_over_time = model.topics_over_time(list(df.text), list(df.year))
fig = model.visualize_topics_over_time(topics_over_time, topics=[37, 86, 18, 67, 82])
fig.write_html("/work/cultural_dynamic.html")

22it [03:47, 10.34s/it]


undefined. 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=90b42492-45fa-4abb-8bb4-ecc1fcb73667' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>